## **Back at it!**
Welcome back to your PINN project! 

*What we have acheived till now:*
*  Made an input ramp function with full flexibility
*  Made a **simple** PINN with input of time, to getdesirable results.
*  Applied hyperarameter tuning to find the best hyperparameters for our task

*What we aim to do now:*
*  Build the next level of PINN with input of Voltage: 
    1.   Use gekko to collect output of capacitor, for different values of maximum amplitude of input voltage
    2.   Now, input parameter to the PINN will be {time, amplitude(anything within a given range specified)}
    3.   The idea is that we want to minimise the use of the ODE solver and allow the PINN to give output standalone!
*  Apply the same process of hyperparamter tuning


Change in PINN taking in 2 inputs
every time points gives one output from PINN. 
Time-Max_amplitude pairs made to extract output

Loss calculated for each row and added together

We are not calculating derivative loss, just normal boundary loss and physics loss

In [ ]:
!pip install gekko
!pip install hyperopt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
#from ramp_input import make_ramp_adv
from gekko import GEKKO
# import optuna
# from optuna.trial import TrialState
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials, space_eval
from hyperopt.pyll.base import scope


# to account for fixed parameters in objective function:
from functools import partial

# if torch.cuda.is_available:
#     device = torch.device('cuda')
# else:
#     device = 'cpu'
device = torch.device('cpu')   


# ---------------------------------------------------------------------------
        This is the making of a ramp function, only for the kaggle notebook

In [ ]:
def make_ramp_adv(time_points, T, t_rise, t_fall, delay, amplitude, last, repetitions):
    '''
    time_points = number of points per time period
    T = time period
    t_rise & t_fall = RATE of rise per unit time and rate for fall respectively
    delay = input delay
    repetitions = how many times you will repeat the input

    Returns: output ramp function (y) and timespace(n) to plot
    '''
    y = []
    n = np.linspace(0, repetitions * T, repetitions * time_points + 1)
    resolution = int(time_points / T)
    rise = t_rise / resolution
    fall = t_fall / resolution
    rise_steps = 0
    
    # delay (zero values)
    for i in range(resolution * delay):
        y.append(0)
    
    # up-ramp values
    for i in range((T - delay) * resolution):
        if rise * i > amplitude:
            break
        y.append(rise * i)
        rise_steps += 1
    
    # amplitude-hold values
    for i in range(last * resolution - rise_steps - int(amplitude // fall) - delay * resolution ):
        y.append(amplitude)
    
    # down-ramp values
    for i in range(int(amplitude // fall)+1):
        y.append(amplitude - fall * i)
    
    # Append zeros to y to match the length of n
    y += [0] * (time_points - len(y))

    y_repeated = y * repetitions
    y_repeated.append(0) #adds one term at the end that makes sure the sizes are matched between n and y_repeated

    return y_repeated, n

# ----------------------------------------------------------------------------------------

In [ ]:
# Making dictionaries for passing into the final function

# For hyperparameter otimisation we need to define a range of hyper parameters (among which Hyperopt will choose)

# Dict 1: Value of parameters to BUILD the ramp input function: CONSTANT
ramp_dict = { 
    "T": 40, # Time period in seconds
    "time_points": 400, # The resolution of each time period: for example, 400 values of Voltage in 40 seconds. Useful in making the linear timespace
    "delay": 0, # The input delay of ramp input in seconds
    "t_rise": 0.5, # the rate of rise of ramp input
    "t_fall": 0.2, # the rate of fall of ramp input
    
    "last": 35, # the point of termination of ramp input in seconds, after which Vin = 0
    "repetitions": 2, # the repetitions for periodic input function
    
    # define the range of maximum amplitudes demanded, for which ramp inputs, Vinput can be stacked. 
    "amp_low": 1, 
    "amp_high": 5
}


# Dict 2: Value of Resistor and capacitor in series R-C circuit
lumped_elements = {
    "R" : 5,
    "C" : 1
}


# Dict 3: Initial conditions for ODE solver. INPUT TO ODE SOLVER (GEKKO)
initial_conditions = {
    "Vc": 5.0,
    "dvdt": 0.0
}


# Dict 4: List of the hyperparameters for training process: TO BE OPTIMISED USING Bayesian Optimisation

# PARAMETER SPACE:
train_dict = {
    
    # Weights to the loss function
    "lambda_boundary": hp.uniform("lambda_boundary", 0.0, 1.0), # Contribution of the boundaries Vc[0] and Vc[-1] to the loss function
    
    "lambda_physics": hp.uniform("lambda_physics", 0.0, 1.0), # Contribution of the physics loss to the loss function
    
    "physics_points": hp.quniform("physics_points", 100, 200, 1), # Number of points where the physics loss is evaluated
    # remember: quniform gives float output by default, so we cast it into integer before using it!
    
    "lambda_deriv": 0, # Contribution of the boundary dvdt[0] to the loss function

    "lr": hp.uniform("lr", 0.0001, 0.01),
    
    "epochs": hp.choice("epochs", [20001, 25001, 30001, 35001, 40001]), # Number of epochs
    
    "n_hidden": hp.quniform("n_hidden", 20, 40, 1),
    "n_layers": hp.quniform("n_layers", 4, 10, 1),
    # typecast the above 2 parameters into INT before using
    
}

In [ ]:
def stack_Vin(ramp_dict):
    
    '''This function repeatedly calls the "make_ramp" function and then stacks the output from each.
        Returns: stacked v_in data and n(timespace)'''
    
    repetitions = ramp_dict["repetitions"]
    time_points = ramp_dict["time_points"]
    T = ramp_dict["T"]
    t_rise = ramp_dict["t_rise"]
    t_fall = ramp_dict["t_fall"]
    amp_low = ramp_dict["amp_low"]
    amp_high = ramp_dict["amp_high"]
    last = ramp_dict["last"]
    delay = ramp_dict["delay"]
    
    # Initialize v_input_data as an empty array for stacking
    v_input_stacked = np.zeros((1, repetitions*time_points+1))  # this is number of timepoints, to define the correct dimensions of the stacked array

    # Loop to create ramp inputs and concatenate along axis 0
    # since loop cannot iterate in float values, we iterate it in the following way: we want a step of 0.1, hence multiply and divide by 10 for this effect
    for i in range(10*amp_low, 10*amp_high + 1, 1):  
        y, n = make_ramp_adv(time_points, T, t_rise, t_rise, delay, i/10, last, repetitions)
        y = np.expand_dims(np.array(y), axis=0)  # Shape (1, 801)
        v_input_stacked = np.concatenate((v_input_stacked, y), axis=0)  # Concatenate along axis 0

    v_input_data = np.delete(v_input_stacked, 0, axis = 0) # the first row is 'zeroes', so delete it

    return v_input_data, n

In [ ]:
# Make a function that can be called repeatedly, that will stack our ground truth for every voltage input

# Assume the voltage_input for one is passed in as input.
def solve_ode(ramp_dict, initial_conditions, lumped_elements, Vin_ramp, n, plot = False):
    # lets find a differential equation for SERIES R C circuit to determine the voltage

    ''' Pre-defined differential equation solved: 
        Input: ramp function parameters, initial conditions, Value of resistor and capacitor, input ramp voltage, n for timespace
        Returns: Voltage across capacitor, derivative of the same, ramp input, linear time space for plotting'''

    
    m = GEKKO()

    # Getting the ramp function
    m.time = n
    
    # make it as parameters to gekko
    Vin = m.Param(value = Vin_ramp)
    
    # make variables here, and put their initial values
    Vc = m.Var(initial_conditions["Vc"]) 
    dvdt = m.Var(initial_conditions["dvdt"])

    R = lumped_elements["R"]
    C = lumped_elements["C"]
    
    # make equation
    m.Equation(dvdt + Vc/(R*C) == Vin/R*C)
    m.Equation(Vc.dt()==dvdt)
    #solve the equation
    m.options.IMODE = 4
    m.solve(disp = False) # if true, then a lot of things will be displayed
    
    DVDT = dvdt.value
    time_space = m.time
    
    # plot the results
    if plot:
        plt.plot(m.time,Vin_ramp,'g:',label='Vin(t)')
        plt.plot(m.time,Vc,'b-',label='Vc(t)')
        plt.plot(m.time, DVDT, 'r--', label='d(Vc(t))/dt')
        plt.ylabel('Vc(t)')
        plt.xlabel('time')
        plt.legend(loc='best')
        plt.show()

    return Vc, DVDT, Vin_ramp, time_space



#-----------------------------------------------------------------------------------------------------

# make the function that calls the above function, and then concatenates and returns the output for us

def stack_solve_Vc(ramp_dict, initial_conditions, lumped_elements, Vin_data, n, plot = False):
    
    repetitions = ramp_dict["repetitions"]
    time_points = ramp_dict["time_points"]
    
    stacked_ode = np.zeros((1, repetitions*time_points+1))
    stacked_ode2 = np.zeros((1, repetitions*time_points+1))
    for Vin in Vin_data:
        
        Vc, DVDT, _, _ = solve_ode(ramp_dict, initial_conditions, lumped_elements, Vin, n, plot = False)
        
        Vc = np.expand_dims(Vc, axis = 0)
        stacked_ode = np.concatenate((stacked_ode, Vc), axis = 0)
        DVDT = np.expand_dims(DVDT, axis = 0)
        stacked_ode2 = np.concatenate((stacked_ode2, DVDT), axis = 0)
        
    stacked_Vc = np.delete(stacked_ode, 0, axis = 0)
    stacked_dvdt = np.delete(stacked_ode2,0, axis = 0)
    
    return stacked_Vc, stacked_dvdt

In [ ]:
v_input_data, n = stack_Vin(ramp_dict)
stacked_Vc, stacked_dvdt = stack_solve_Vc(ramp_dict, initial_conditions, lumped_elements, v_input_data, n)

In [ ]:
plt.subplots(1,1)
plt.plot(n, v_input_data[40])
plt.plot(n, stacked_Vc[40])

# Make a complete pipeline to show the model's flexibility:
### Objectives: 
1. Incorporate the initial conditions into the model.
2. Include the flexibility of your ramp input function, add the params (make a dictionary)
3. Display the ground truth values from GEKKO as a part of training process

Path: 
Ramp input -> GEKKO -> Voltage curve -> Model_train -> Output

In [ ]:
# the definition of the PINN remains the same, because the N_inputs tab previously held value = 1, now it will hold value 2.
# basically as our model becomes larger, with more parameters, we can keep on increasing the number of inputs

# Number of inputs: number of features and values to be considered in PINN and solving

class FCN(nn.Module):
    """Defines a standard fully-connected network in PyTorch.
    Number of inputs, Number of outputs, Number of hidden inputs, Total layers"""
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(nn.Linear(N_INPUT, N_HIDDEN), activation())
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x



In [ ]:
# Function to extract the boundary values of all the stacked ground truth items
def extract_boundaries(stacked_vc, stacked_dvdt):

    '''This function is responsible for extracting the boundaries of the solved ODE equation
        Shape of stacked_vc is: number of discrete amplitudes, number of timepoints
        Returned datatype: Dictionary
        These boundary values will be used in the Loss function of the PINN'''
    
    boundaries = {
        "u_last" : stacked_vc[:,-1],
        "u_first" : stacked_vc[:,0],
        "du_first" : stacked_dvdt[:, 0],
        "du_last" : stacked_dvdt[:, -1]
    }
    return boundaries

In [ ]:
# in my earlier implementation, we had a single row which was interpolated to calculate the physics loss
# now we are dealing with multiple input functions at once so we need a code to interpolate them all together

def interpolate_multiple_rows(v_input, physics_points):
    interpolated_inputs = []
    
    # Iterate over each row in v_input
    for row in v_input:
        # Interpolate each row
        interpolated_row = F.interpolate(torch.tensor(row, dtype=torch.float32).view(1, 1, -1), size=physics_points).view(-1, 1)
        # Enable gradients
        interpolated_row.requires_grad_(True)
        # Store the interpolated row
        interpolated_inputs.append(interpolated_row)
    
    # Stack the rows back into a single tensor. If we don't do this, then they will appear as separate tensors
    interpolated_inputs = torch.stack(interpolated_inputs)

    return interpolated_inputs

In [ ]:
def train_model(lr, lumped_elements, boundaries, v_input_data, stacked_vc, lambda_boundary, lambda_deriv, lambda_physics, epochs, physics_points, 
                n_hidden, n_layers, n_repetitions, time_period, time_points, amp_low, amp_high):

    '''
    Inputs: lr --> learning rate
            Boundaries --> Final and initial values of capacitor voltage
            v_input --> Input ramp function
            Vc --> ground truth from GEKKO
            lambda_boundary/deriv/physics --> contributory weights of the boundary loss, derivative boundary loss, and physics loss
            epochs --> number of epochs
            physics_points --> number of points for which physics loss is evaluated
            n_hidden + n_layers --> definition of number of nodes per layer and number of layers in PINN model
            n_repetitions --> number of repetitions of ramp input
            time_period + time_points --> time period of ramp input in seconds and number of time_points per period
            
    Objective of function: Instantiating the PINN model; building the loss function with the mentioned weights; '''
    
    #torch.manual_seed(123)
    # Extracting standard values from dictionaries
    
    R = lumped_elements["R"]
    C = lumped_elements["C"]
    u_first = boundaries["u_first"] # these are arrays of boundary conditions for every case
    u_last = boundaries["u_last"]
    du_first = boundaries["du_first"]
    du_last = boundaries["du_last"]

    # Define a fully connected network (FCN) for the PINN. 2 inputs include time and amplitude of our ramp input function
    pinn = FCN(2, 1, n_hidden, n_layers)
    pinn.to(device)
    
    # Define boundary points for the boundary loss (time points, not voltage values)
    time_point_left = torch.tensor([0.0], dtype=torch.float32).view(-1, 1).requires_grad_(True)  
    time_point_right = torch.tensor([time_points*n_repetitions], dtype=torch.float32).view(-1, 1).requires_grad_(True)  # the last value in the time series
    time_point_left.to(device)
    time_point_right.to(device)
    
    # Defining linear timespace with number of points = 'physics_points'; TO CALCULATE PHYSICS LOSS
    t_physics = torch.linspace(0, time_period*n_repetitions, physics_points, dtype=torch.float32).view(-1, 1).to(device).requires_grad_(True)
    
    # Making V_input smaller in size to match number of physics points; and enabling grad. Call interpolate function
    # v_input_interp = interpolate_multiple_rows(v_input_data, physics_points)
    
    # Initialize the optimizer
    optimiser = torch.optim.Adam(pinn.parameters(), lr= lr)

    
    t_test = torch.linspace(0, time_period*n_repetitions, time_points*n_repetitions+1).view(-1,1).to(device)

    # Converting the ground truth from the ODE solver into a tensor
    u_exact = torch.tensor(stacked_vc).to(device) # exact solution(s) from GEKKO
    
    # making a range of amplitude for input to the PINN:
    amplitude_range = torch.arange(amp_low, amp_high + 0.1, 0.1).view(-1, 1).to(device)
    
    # Training loop
    for i in range(epochs):
        optimiser.zero_grad()
        total_loss = 0
        
        # We calculate loss for every row in the data, and we directly add them together
        # Changes to PINN: we need to concatenate the time and the amplitude inputs.
        
        for row in range(v_input_data.shape[0]):
            
            v_input_row = v_input_data[row]  # Extract one row of v_input_data
           
            u_exact_row = u_exact[row]  # Corresponding ground truth
         
            amplitude_value = amplitude_range[row]
            amplitude_value.to(device)
            
            # Interpolate the row for the physics loss
            v_input_interp = F.interpolate(torch.tensor(v_input_row, dtype=torch.float32).view(1, 1, -1), size=physics_points).view(-1, 1).requires_grad_(True)
            v_input_interp.to(device)
            
            # Physics inputs: concatenate time and amplitude. Extend the single amplitude value to make pairs of time and amp
            time_amplitude_input = torch.cat([t_physics, amplitude_value.expand(physics_points, -1)], dim=1).to(device)
            
            
            # Compute boundary loss: (predicted values from PINN - ground truth)^2
            u_left = pinn(torch.cat([time_point_left, amplitude_value.view(1,1)], dim = 1)).to(device)  # at t = 0
            
            loss1a = (torch.squeeze(u_left) - u_first[row])**2  
            loss1a.to(device)

            u_right = pinn(torch.cat([time_point_right, amplitude_value.view(1,1)], dim = 1)).to(device)  # at t = end
            loss1b = (torch.squeeze(u_right) - u_last[row])**2  
            loss1b.to(device)

            # loss for time derivative at the initial condition
#             dudt_left = torch.autograd.grad(u_left, time_point_left, torch.ones_like(u_left), create_graph=True)[0]
#             loss_deriv = (torch.squeeze(dudt_left) - du_first)**2

            # Physics loss: differential equation with input incorporated. Loss calculated at multiple points. Thus we need to expand the 
            # amplitude value across the time points, so that we have pairs of time vs max_amplitude(same for all time points)
                          
            u_physics = pinn(time_amplitude_input)  # Evaluate PINN at physics points
            u_physics.to(device)
            dudt_physics = torch.autograd.grad(u_physics, t_physics, torch.ones_like(u_physics), create_graph=True)[0]
            dudt_physics.to(device)
            loss_physics = torch.mean((dudt_physics + u_physics / (R * C) - v_input_interp / (R * C))**2).to(device)

            # Total loss
            row_loss = lambda_boundary * (loss1a + loss1b) + lambda_physics * loss_physics
            row_loss.to(device)
            total_loss += row_loss
            total_loss.to(device)
            
            
        total_loss.backward()  
        optimiser.step()  
        
        # Plot the result during training
#         if i % 10000 == 0:
#             u_test = pinn(t_test).detach()
#             plt.figure(figsize=(6, 2.5))
#             plt.scatter(t_physics.detach()[:, 0], torch.zeros_like(t_physics)[:, 0], s=20, lw=0, color="tab:green", alpha=0.6)
#             plt.scatter(time_point_left.detach()[:, 0], torch.zeros_like(time_point_left)[:, 0], s=20, lw=0, color="tab:red", alpha=0.6)
#             plt.plot(t_test[:, 0], u_exact[:, 0], label="Exact solution", color="tab:grey", alpha=0.6)
#             plt.plot(t_test[:, 0], u_test[:, 0], label="PINN solution", color="tab:green")
#             plt.title(f"Training step {i}")
#             plt.legend()
#             plt.show()

    return total_loss, pinn


In [ ]:
def train_model(lr, lumped_elements, boundaries, v_input_data, stacked_vc, lambda_boundary, lambda_deriv, lambda_physics, epochs, physics_points, 
                n_hidden, n_layers, n_repetitions, time_period, time_points, amp_low, amp_high):

    R = lumped_elements["R"]
    C = lumped_elements["C"]
    u_first = boundaries["u_first"]
    u_last = boundaries["u_last"]

    # Define the fully connected network (FCN)
    pinn = FCN(2, 1, n_hidden, n_layers).to(device)

    # Define boundary points for the boundary loss
    time_point_left = torch.tensor([0.0], dtype=torch.float32).view(-1, 1).to(device).requires_grad_(True)
    time_point_right = torch.tensor([time_points * n_repetitions], dtype=torch.float32).view(-1, 1).to(device).requires_grad_(True)

    # Define linear time space for physics loss
    t_physics = torch.linspace(0, time_period * n_repetitions, physics_points, dtype=torch.float32).view(-1, 1).to(device).requires_grad_(True)

    # Initialize the optimizer
    optimiser = torch.optim.Adam(pinn.parameters(), lr=lr)

    # Convert the ground truth from the ODE solver into a tensor
    u_exact = torch.tensor(stacked_vc).to(device)

    # Create a range of amplitudes for input to the PINN
    amplitude_range = torch.arange(amp_low, amp_high + 0.1, 0.1).view(-1, 1).to(device)

    # Training loop
    for i in range(epochs):
        optimiser.zero_grad()
        total_loss = 0

        for row in range(v_input_data.shape[0]):
            v_input_row = v_input_data[row]
            u_exact_row = u_exact[row]

            amplitude_value = amplitude_range[row].to(device)

            # Interpolate the row for the physics loss
            v_input_interp = F.interpolate(torch.tensor(v_input_row, dtype=torch.float32).view(1, 1, -1).to(device), size=physics_points).view(-1, 1).requires_grad_(True)

            # Create time and amplitude input pairs
            time_amplitude_input = torch.cat([t_physics, amplitude_value.expand(physics_points, -1)], dim=1).to(device)

            # Compute boundary loss
            u_left = pinn(torch.cat([time_point_left, amplitude_value.view(1, 1)], dim=1)).to(device)
            loss1a = (torch.squeeze(u_left) - u_first[row]) ** 2

            u_right = pinn(torch.cat([time_point_right, amplitude_value.view(1, 1)], dim=1)).to(device)
            loss1b = (torch.squeeze(u_right) - u_last[row]) ** 2

            # Physics loss
            u_physics = pinn(time_amplitude_input).to(device)
            dudt_physics = torch.autograd.grad(u_physics, t_physics, torch.ones_like(u_physics), create_graph=True)[0].to(device)
            loss_physics = torch.mean((dudt_physics + u_physics / (R * C) - v_input_interp / (R * C)) ** 2).to(device)

            # Total loss
            row_loss = lambda_boundary * (loss1a + loss1b) + lambda_physics * loss_physics
            total_loss += row_loss

        total_loss.backward()
        optimiser.step()

    return total_loss, pinn

## 

In [ ]:
def save_model(str):

    '''Save the model with annotation of your choice, to your default folder'''
    output_model_file = '/kaggle/working/RC_PINN_'+str+'.pt'
    
    model_to_save = FCN
    torch.save(model_to_save, output_model_file)

In [ ]:
def RAW_solve_train_objective(train_dict, ramp_dict, initial_conditions, lumped_elements, v_input_data, stacked_Vc):

    '''An end-to-end function for deployment:
    Inputs: ramp_dict --> parameters to define the input ramp function
            initial_conditions --> initial conditions for solving the ODE equation
            train_dict --> hyperparameters for the training process
            
    returns: predicted voltage across capacitor'''
    
#     # stacked input voltage
#     v_input_data, n = stack_Vin(ramp_dict)
    
#     # stacked outout capacitor voltage
#     stacked_Vc, stacked_dvdt = stack_solve_Vc(ramp_dict, initial_conditions, lumped_elements, v_input_data, n)
    
    # extract boundaries to compute the boundary loss in 'train_model'
    boundaries = extract_boundaries(stacked_Vc, stacked_dvdt)

    # compute and retrieve the predicted values from the training process
    loss, pinn = train_model(train_dict["lr"], lumped_elements, boundaries, v_input_data, stacked_Vc, train_dict["lambda_boundary"], train_dict["lambda_deriv"], 
                                     train_dict["lambda_physics"],train_dict["epochs"], int(train_dict["physics_points"]), int(train_dict["n_hidden"]), int(train_dict["n_layers"]), 
                                     ramp_dict["repetitions"], ramp_dict["T"], ramp_dict["time_points"], ramp_dict["amp_low"], ramp_dict["amp_high"])

    # save the model
    #save_model(str)
    return {'loss': loss,
            'status': STATUS_OK,
            'model': pinn,
            'params': train_dict}

In [ ]:
save_train_objective = partial(RAW_solve_train_objective, 
                              ramp_dict = ramp_dict,
                              initial_conditions = initial_conditions,
                              lumped_elements = lumped_elements,
                              v_input_data = v_input_data,
                              stacked_Vc = stacked_Vc)

In [ ]:
trials = Trials()


best_params = fmin(
    fn=save_train_objective,
    space=train_dict,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials)

In [ ]:
save_model('level_2_attempt3')

In [ ]:
strin = 'level_2_attempt3'
try: 
	geeky_file = open('best_parameters'+ strin +'.txt', 'a') 
	geeky_file.write(str(best_params)) 
	geeky_file.close() 

except: 
	print("Unable to append to file")


In [ ]:
try: print(best_params)

except: print('best_params not found')